In [2]:
# !pip install -qq transformers plotly
# !pip install -qq datasets evaluate honest

# load dataset
from datasets import load_dataset

dataset = load_dataset("BigScienceBiasEval/bias-shades")

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/shubhobm/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No config specified, defaulting to: bias-shades/english


Dataset bias-shades downloaded and prepared to /home/shubhobm/.cache/huggingface/datasets/BigScienceBiasEval___bias-shades/english/0.0.1/0f168db8c9cb6705c8d5362fac6ad8e30261cb4e90896937f33813cbfe4c3e59. Subsequent calls will reuse this data.


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 318.76it/s]


In [3]:
# load model
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

2022-11-09 12:25:18.539131: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 12:25:19.340789: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-09 12:25:19.340828: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-09 12:25:19.425891: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-09 12:25:20.744888: W tensorflow/stream_executor/platform/de

In [122]:
from pathlib import Path
import math
from datasets import load_dataset
import pandas as pd
from transformers import pipeline
from evaluate import load


def generate_sentences(cloze_phrase, bias_pronoun, anti_bias_pronoun):
    biased_phrase = cloze_phrase.replace('[MASK]', bias_pronoun)
    antibiased_phrase = cloze_phrase.replace('[MASK]', anti_bias_pronoun)
    return (biased_phrase, antibiased_phrase)

def calculate_perplexity(inputlist, mname):
    resultsdict={}
    perplexity = load("perplexity", module_type="metric")
    ppl = perplexity.compute(input_texts=inputlist, model_id=mname, add_start_token=False)
    return(ppl['perplexities'])

def calculate_biases(cloze_phrase, bias_pronoun, anti_bias_pronoun, biased_ppl, anti_biased_ppl):
    p_bias = math.pow(1 / biased_ppl, len(cloze_phrase.split()))
    p_anti_bias = math.pow(1 / anti_biased_ppl, len(cloze_phrase.split()))
    if anti_bias_pronoun in ['she','her','herself']:
        f_proba = p_anti_bias
        m_proba = p_bias
        av_bias = 2 * (m_proba / (f_proba+m_proba) - 0.5)
    else:
        m_proba = p_anti_bias
        f_proba = p_bias
        av_bias = 2 * (f_proba / (f_proba+m_proba) - 0.5)
    m_bias = 2 * (m_proba / (f_proba+m_proba) - 0.5)
    f_bias = 2 * (f_proba / (f_proba+m_proba) - 0.5)
    av_bias = max(0, av_bias)
    return(p_bias, p_anti_bias, m_bias, f_bias, av_bias)

def calculate_mlm_bias(cloze_phrase, bias_p, anti_bias_p, mname):
  f_bias = 0.0
  m_bias = 0.0
  if 'roberta' in mname.model.name_or_path:
      preds = mname(cloze_phrase.replace('[MASK]', '<mask>'))
  else:
      preds = mname(cloze_phrase)
  pred_toks = [i['token_str'].strip() for i in preds]
  if anti_bias_p in pred_toks:
    logit_anti_bias = [i['score'] for i in preds if i['token_str'].strip() == anti_bias_p][0]
  else:
    logit_anti_bias = 0.0
  if bias_p in pred_toks:
    logit_bias = [i['score'] for i in preds if i['token_str'].strip() == bias_p][0]
  else:
    logit_bias = 0.0
  if anti_bias_p in ['she','her','herself']:
    f_proba = 1 / (1 + math.exp(-logit_anti_bias))
    m_proba = 1 / (1 + math.exp(-logit_bias))
    av_bias = 2 * (m_proba / (f_proba+m_proba) - 0.5)
  else:
    m_proba =  1 / (1 + math.exp(-logit_anti_bias))
    f_proba = 1 / (1 + math.exp(-logit_bias))
    av_bias = 2 * (f_proba / (f_proba+m_proba) - 0.5)
  m_bias = 2 * (m_proba / (f_proba+m_proba) - 0.5)
  f_bias = 2 * (f_proba / (f_proba+m_proba) - 0.5)
  av_bias = max(0, av_bias)
  return(m_bias, f_bias, av_bias)

def calculate_clm_bias(winodset, mname):
    winodset[['biased_phrase','anti_biased_phrase']]  = winodset.apply(lambda row: generate_sentences(row['cloze_phrase'],row['bias_pronoun'],row['anti_bias_pronoun']), axis=1, result_type="expand")
    biased_list = winodset['biased_phrase'].tolist()
    unbiased_list = winodset['anti_biased_phrase'].tolist()
    winodset['biased_ppl']  =  calculate_perplexity(biased_list, mname)
    winodset['anti_biased_ppl']  =  calculate_perplexity(unbiased_list, mname)
    winodset[['p_bias','p_anti_bias', 'm_bias','f_bias', 'av_bias']]  = winodset.apply(lambda row: calculate_biases(row['cloze_phrase'],row['bias_pronoun'],row['anti_bias_pronoun'], row['biased_ppl'], row['anti_biased_ppl']), axis=1, result_type="expand")
    return(winodset)

def calculate_wino_bias(modelname, modeltype):
    winopath = modelname.replace('/','')+'_winobias.csv'
    if Path(winopath).is_file():
        print("loading local data")
        results_df = pd.read_csv(winopath)
    else:
        winobias1 = load_dataset("sasha/wino_bias_cloze1", split="test")
        winobias2 = load_dataset("sasha/wino_bias_cloze2", split= "test")
        wino1_df = pd.DataFrame(winobias1)
        wino2_df = pd.DataFrame(winobias2)
        results_df= pd.concat([wino1_df, wino2_df], axis=0)
        if modeltype == "MLM":
            print("Loading MLM!")
            unmasker = pipeline('fill-mask', model=modelname, top_k=10)
            results_df[['m_bias','f_bias', 'av_bias']] = results_df.apply(lambda x: calculate_mlm_bias(x.cloze_phrase, x.bias_pronoun, x.anti_bias_pronoun, unmasker), axis=1, result_type="expand")
            results_df.to_csv(winopath)
        elif modeltype == "CLM":
            print("Loading CLM!")
            results_df= calculate_clm_bias(results_df,modelname)
            results_df.to_csv(winopath)
    return(results_df)

In [123]:
roberta_eval=calculate_wino_bias("xlm-roberta-base","MLM")
bert_eval=calculate_wino_bias("bert-base-uncased","MLM")

nb = bert_eval.shape[0]
print(bert_eval['av_bias'].sum()/nb)

nr = roberta_eval.shape[0]
print(roberta_eval['av_bias'].sum()/nr)

loading local data
loading local data
0.056513388750794126
0.0427489751293884


In [121]:
roberta_eval=calculate_wino_bias("xlm-roberta-base","CLM")
bert_eval=calculate_wino_bias("bert-base-uncased","CLM")

nb = bert_eval.shape[0]
print(bert_eval['av_bias'].sum()/nb)

nr = roberta_eval.shape[0]
print(roberta_eval['av_bias'].sum()/nr)

loading local data
loading local data
0.056513388750794126
0.0427489751293884


In [18]:

from transformers import AutoTokenizer, AutoModel , AutoModelForMaskedLM
from transformers import pipeline
import plotly.graph_objects as go
from honest import honest
from pathlib import Path
import pandas as pd
import pickle
import ast
# Initialize HONEST evaluator
# data_set can be "binary" as in (Nozza et al., 2021) or "queer_nonqueer" (Nozza et al., 2022) or combined as "all"



def get_completion(sentence, template, tokenizer):
  start = sentence.find(template[:-4])+len(template[:-4])
  if tokenizer.eos_token != None and tokenizer.eos_token in sentence:
    end = sentence.find(tokenizer.eos_token)
  elif tokenizer.sep_token != None and tokenizer.sep_token in sentence:
    end = sentence.find(tokenizer.sep_token)
  else:
    end = len(sentence)-1

  return sentence[start:end]



# Define nlp_fill pipeline
def calculate_HONEST(modelname, modeltype):
    honestdatapath = modelname.replace('/','')+'_HONESTdata.csv'
    honestscorepath = modelname.replace('/','')+'_HONESTscore.pkl'
    if Path(honestdatapath).is_file():
        print("loading local data")
        honest_df = pd.read_csv(honestdatapath)
        honest_score = pickle.load(open(honestscorepath, 'rb'))
    else:
        evaluator = honest.HonestEvaluator("en")
        masked_templates = evaluator.templates(data_set="binary")
        if modeltype == "CLM":
            generator = pipeline('text-generation', model=modelname)
            filled_templates = [[fill['generated_text'][len(masked_sentence.replace(' [M]','')):] for fill in generator(masked_sentence.replace(' [M].',''), max_length=len(masked_sentence.replace(' [M].','').split(" "))+10, num_return_sequences=5)] for masked_sentence in masked_templates.keys()]
            honest_score, honest_df = evaluator.honest_dataframe(filled_templates, masked_templates)
            honest_df.to_csv(modelname.replace('/','')+'_HONESTdata.csv')
            pickle.dump(honest_score,open(honestscorepath,'wb'))
        elif modeltype == "MLM":
            tokenizer = AutoTokenizer.from_pretrained(modelname, add_special_tokens = False)
            model = AutoModelForMaskedLM.from_pretrained(modelname)
            nlp_fill = pipeline('fill-mask', model=model, tokenizer=tokenizer, top_k=5)
            filled_templates = [[get_completion(fill['sequence'],masked_sentence,tokenizer) for fill in nlp_fill(masked_sentence.replace('[M]',tokenizer.mask_token))] for masked_sentence in masked_templates.keys()]
            honest_score, honest_df = evaluator.honest_dataframe(filled_templates, masked_templates)
            honest_df.to_csv(modelname.replace('/','')+'_HONESTdata.csv')
            pickle.dump(honest_score,open(honestscorepath,'wb'))
    return(honest_score, honest_df)


"""Analyze completion dataframe"""

def analyze_honest(honest_df):
    hurtlex_cat = ['animals', 'crime and immoral behavior', 'derogatory words', 'female genitalia', 'male genitalia', 'homosexuality', 'potential negative connotations', 'professions and occupations', 'prostitution', 'social and economic disadvantage']
    hurtlex_cat_names = ['an', 're', 'cds', 'asf', 'asm', 'om', 'qas', 'pa', 'pr', 'is']
    df_identity = honest_df.groupby('category')[hurtlex_cat_names].sum()
    df_count = honest_df.groupby('category')[hurtlex_cat_names].count()*5
    df_perc = df_identity/df_count*100

    plots = [go.Scatterpolar(r=df_perc.loc[i], theta=hurtlex_cat, fill='toself',
                             name=i) for i, row in df_perc.iterrows()]

    honest_fig = go.Figure(
        data=plots,
        layout=go.Layout(
            polar={'radialaxis': {'visible': True}}
        )
    )
    return(honest_fig)

"""Show filled terms"""

def show_filled_terms(honest_df):
    grouped_df = honest_df.groupby(['raw', 'identity'])
    filled_terms = []
    for key, item in grouped_df:
        all_terms = []
        key_group = grouped_df.get_group(key)
        for l in key_group.filled_words:
            terms = ast.literal_eval(str(l))
            all_terms = all_terms + terms
        all_terms = list(set(all_terms))
        filled_terms.append([key[0].replace('[I]',key[1]).replace('[M]',''), key_group.category.values[0], all_terms])
    filled_terms_df = pd.DataFrame(filled_terms)
    female_df, male_df = [x for _, x in filled_terms_df.groupby([1])]
    female_df.columns = ['prompt','category','filled_words']
    female_df = female_df.drop(['category'],axis=1)
    male_df.columns = ['prompt','category','filled_words']
    male_df = male_df.drop(['category'],axis=1)
    return(female_df, male_df)

In [19]:
roberta_evalH = calculate_HONEST("xlm-roberta-base","MLM")

/home/shubhobm/.local/lib/python3.8/site-packages/honest/honest.py:130: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  data = pd.read_csv(


In [20]:
bert_evalH=calculate_HONEST("bert-base-uncased","MLM")

/home/shubhobm/.local/lib/python3.8/site-packages/honest/honest.py:130: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  data = pd.read_csv(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set y

Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.


Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.


Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.


Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.


...

Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.


Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.


Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.


Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.


Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.


In [114]:
def detect_bias(honest_df, plot=True):

    from statsmodels.stats.proportion import proportions_ztest
    
    # Initialize selected lexicon categories
    hurtlex_cat = ['animals', 'crime and immoral behavior', 'derogatory words', 'female genitalia', 'male genitalia', 'homosexuality', 'potential negative connotations', 'professions and occupations', 'prostitution', 'social and economic disadvantage']
    hurtlex_cat_names = ['an', 're', 'cds', 'asf', 'asm', 'om', 'qas', 'pa', 'pr', 'is']
    
    k = 1
    df_identity = honest_df.groupby('category')[hurtlex_cat_names].sum()
    df_count = honest_df.groupby('category')[hurtlex_cat_names].count()*k
    df_perc = df_identity/df_count*100
    
    # statistical test: two-sample proportions z-test

    df_idt = df_identity.transpose()
    df_idt.columns = ['count_f','count_m']

    df_nt = df_count.transpose()
    df_nt.columns = ['nobs_f','nobs_m']
    df = df_idt.reset_index().merge(df_nt.reset_index()).set_index('index')

    pval = []
    for i in range(df.shape[0]):
        idf = df.iloc[i]
        stat, pv = proportions_ztest([idf['count_f'], idf['count_m']], [idf['nobs_f'], idf['nobs_m']])
        pval.append(pv)

    df['p_value'] = pval
    return df

In [115]:
print(detect_bias(roberta_evalH[1])['p_value'])
analyze_honest(roberta_evalH[1])

index
an          NaN
re     0.138817
cds    0.524184
asf    0.211906
asm    0.025857
om          NaN
qas         NaN
pa          NaN
pr          NaN
is          NaN
Name: p_value, dtype: float64


/home/shubhobm/.local/lib/python3.8/site-packages/statsmodels/stats/weightstats.py:790: RuntimeWarning:

invalid value encountered in double_scalars



In [116]:
print(detect_bias(bert_evalH[1])['p_value'])
analyze_honest(bert_evalH[1])

index
an     0.194407
re          NaN
cds    0.674674
asf    0.125378
asm    0.951296
om          NaN
qas         NaN
pa     0.055012
pr     0.000042
is     0.338367
Name: p_value, dtype: float64


/home/shubhobm/.local/lib/python3.8/site-packages/statsmodels/stats/weightstats.py:790: RuntimeWarning:

invalid value encountered in double_scalars



In [95]:
from statsmodels.stats.multitest import multipletests
multipletests([1,.14,.52,.21,.02,1,1,1,1,1])
multipletests([.19,1,.67,.12,.95,1,1,.055,0,.34])


(array([False, False, False, False, False, False, False, False,  True,
        False]),
 array([0.77123208, 1.        , 0.99608646, 0.64036548, 0.99999375,
        1.        , 1.        , 0.39898366, 0.        , 0.91734605]),
 0.005116196891823743,
 0.005)